# SummaryExtractor

In [1]:
%%time

from llama_index.core.extractors import (
    SummaryExtractor,
)

CPU times: user 2.53 s, sys: 352 ms, total: 2.88 s
Wall time: 2.53 s


In [2]:
%%time

from llama_index.llms.openai_like import OpenAILike
from llama_index.core import Settings

llm = OpenAILike(model="qwen2-7b-6k", 
                 api_base="http://192.168.0.72:3000/v1", 
                 api_key="sk-bJP6QSnUfjAYeYeE505d3eBf63A643BeB0B8E350Df9b7750",
                 is_chat_model=True,
                 temperature=0.1
                )
Settings.llm=llm

CPU times: user 121 ms, sys: 8.08 ms, total: 129 ms
Wall time: 128 ms


In [4]:
%%time

from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader(input_files=["../content-short.txt"]).load_data()

CPU times: user 1.37 ms, sys: 0 ns, total: 1.37 ms
Wall time: 1.14 ms


In [15]:
%%time

from llama_index.core.node_parser import SentenceSplitter

node_parser = SentenceSplitter(chunk_size=1024, chunk_overlap=20)

CPU times: user 77 µs, sys: 12 µs, total: 89 µs
Wall time: 91.1 µs


## 默认总结当前节的摘要

In [16]:
%%time

from llama_index.core.schema import MetadataMode
from llama_index.core.ingestion import IngestionPipeline

extrator=SummaryExtractor()

pipeline = IngestionPipeline(
    transformations=[
        node_parser, 
        extrator
    ])

CPU times: user 180 µs, sys: 0 ns, total: 180 µs
Wall time: 182 µs


In [17]:
%%time

import nest_asyncio
nest_asyncio.apply()

nodes = pipeline.run(
    documents=documents,
    in_place=False,
    show_progress=True,
)

Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

100% 5/5 [00:15<00:00,  3.10s/it]

CPU times: user 90.4 ms, sys: 4.69 ms, total: 95.1 ms
Wall time: 15.5 s


In [19]:
len(nodes)

5

In [14]:
nodes[3].metadata

{'file_path': '../content-short.txt',
 'file_name': 'content-short.txt',
 'file_type': 'text/plain',
 'file_size': 11342,
 'creation_date': '2024-06-08',
 'last_modified_date': '2024-06-08',
 'section_summary': '本节主要讨论了以下关键内容：\n\n1. **文件信息**：提及了一个名为`content-short.txt`的文本文件，该文件位于`../content-short.txt`路径下。文件类型为纯文本（text/plain），大小为11342字节，创建日期和最后修改日期均为2024年6月8日。\n\n2. **会议目的**：召开此次会议的主要目的是吸引中国企业的投资。这表明会议旨在促进经济合作，并特别关注与中国企业之间的互动与投资机会。\n\n3. **会议内容**：会议的主旨和议题被明确指定为几个细分领域，这些领域都与高科技产业相关联。这暗示了会议将聚焦于讨论和展示在高科技领域的合作潜力、项目和技术交流等。\n\n4. **经济合作前提**：强调了双边牢固的外交关系是实现经济合作的前提条件。这表明会议不仅关注经济层面的合作，还重视政治和外交基础的重要性，以确保双方都能从合作中获得互惠互利的结果。\n\n综上所述，本节主要围绕一个旨在吸引中国投资的会议进行讨论，重点关注高科技产业领域的合作，并强调了在经济合作之前建立牢固双边关系的重要性。'}

## 可以设置增加上一节和下一节的摘要

In [21]:
%%time

extrator=SummaryExtractor(summaries=['self', 'prev', 'next'])

pipeline = IngestionPipeline(
    transformations=[
        node_parser, 
        extrator
    ])

CPU times: user 199 µs, sys: 0 ns, total: 199 µs
Wall time: 202 µs


In [22]:
%%time

nodes = pipeline.run(
    documents=documents,
    in_place=False,
    show_progress=True,
)

Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

100% 5/5 [00:15<00:00,  3.18s/it]

CPU times: user 89.2 ms, sys: 7.77 ms, total: 96.9 ms
Wall time: 15.9 s


In [23]:
len(nodes)

5

In [24]:
nodes[3].metadata

{'file_path': '../content-short.txt',
 'file_name': 'content-short.txt',
 'file_type': 'text/plain',
 'file_size': 11342,
 'creation_date': '2024-06-08',
 'last_modified_date': '2024-06-08',
 'prev_section_summary': '本段内容主要围绕中巴关系、中国模式对巴基斯坦的借鉴、印度与周边国家的关系以及阿富汗局势对中巴关系的影响等几个关键主题展开。核心实体包括：\n\n1. **中巴关系**：讨论了中巴经济走廊的安全问题和反恐合作，强调恐怖主义作为国际现象对中巴关系的影响。\n2. **中国模式**：提及中国模式对巴基斯坦的借鉴作用，特别是通过多次访华经历对中国深入了解后，希望在富强之路上得到中国的帮助。\n3. **印度与周边国家的关系**：分析了印度与其邻国如尼泊尔、孟加拉国、缅甸、马尔代夫等之间的紧张关系和领土争端，以及印度的民族主义情绪对穆斯林群体的影响。\n4. **阿富汗局势**：强调了阿富汗政治生态的修复对于稳定中巴关系的重要性，以及其作为四战之地的历史背景对中巴商贸对接可能产生的影响。\n\n这些主题和实体共同构成了关于地区外交、国家间合作与冲突、以及国际安全问题的一系列讨论。',
 'next_section_summary': '本节主要讨论了以下关键主题和实体：\n\n1. **巴基斯坦代表团访问中国**：去年12月15日，巴基斯坦电看守商务部长戈哈尔·埃贾兹率领代表团访问了中国。这一活动可能涉及经济合作、贸易交流等议题。\n\n2. **印度的工业化项目与自主可控制造业**：文章提到印度在推动各种工业项目，并强调制造业的自主性。同时指出印度面临极端天气影响，尤其是热浪导致的水电能源短缺问题，这不仅影响民用也波及工业领域。\n\n3. **印度与巴基斯坦面临的共同挑战**：讨论了两国在能源供应方面可能遇到的问题，特别是由于极端天气和工业污染带来的挑战。文章还提到了长期来看这些因素对印度经济的不利影响。\n\n4. **印度的历史外交策略**：分析了印度过去与苏联、现在与美国之间的关系变化，以及这种外交策略如何影响其国内政策和国际地位。\n\n

In [25]:
nodes[3].get_content(metadata_mode="all")

'[Excerpt from document]\nfile_path: ../content-short.txt\nfile_name: content-short.txt\nfile_type: text/plain\nfile_size: 11342\ncreation_date: 2024-06-08\nlast_modified_date: 2024-06-08\nprev_section_summary: 本段内容主要围绕中巴关系、中国模式对巴基斯坦的借鉴、印度与周边国家的关系以及阿富汗局势对中巴关系的影响等几个关键主题展开。核心实体包括：\n\n1. **中巴关系**：讨论了中巴经济走廊的安全问题和反恐合作，强调恐怖主义作为国际现象对中巴关系的影响。\n2. **中国模式**：提及中国模式对巴基斯坦的借鉴作用，特别是通过多次访华经历对中国深入了解后，希望在富强之路上得到中国的帮助。\n3. **印度与周边国家的关系**：分析了印度与其邻国如尼泊尔、孟加拉国、缅甸、马尔代夫等之间的紧张关系和领土争端，以及印度的民族主义情绪对穆斯林群体的影响。\n4. **阿富汗局势**：强调了阿富汗政治生态的修复对于稳定中巴关系的重要性，以及其作为四战之地的历史背景对中巴商贸对接可能产生的影响。\n\n这些主题和实体共同构成了关于地区外交、国家间合作与冲突、以及国际安全问题的一系列讨论。\nnext_section_summary: 本节主要讨论了以下关键主题和实体：\n\n1. **巴基斯坦代表团访问中国**：去年12月15日，巴基斯坦电看守商务部长戈哈尔·埃贾兹率领代表团访问了中国。这一活动可能涉及经济合作、贸易交流等议题。\n\n2. **印度的工业化项目与自主可控制造业**：文章提到印度在推动各种工业项目，并强调制造业的自主性。同时指出印度面临极端天气影响，尤其是热浪导致的水电能源短缺问题，这不仅影响民用也波及工业领域。\n\n3. **印度与巴基斯坦面临的共同挑战**：讨论了两国在能源供应方面可能遇到的问题，特别是由于极端天气和工业污染带来的挑战。文章还提到了长期来看这些因素对印度经济的不利影响。\n\n4. **印度的历史外交策略**：分析了印度过去与苏联、现在与美国之间的关系变化，以及这种外交策略如何影响其国内政策和国际地位。\n\n5. **巴基斯坦的政

In [26]:
nodes[3].get_content()

'恐怖分子是我们共同的敌人，所以我们应该要统一起来，齐心合力面对解决。\n\n中巴关系目前在各个领域的发展势头都很好，比如在政治、经济、贸易、国防、教育、科技各个领域等等，在处理恐怖分子的问题上，双方也有巨大的合作空间。两个国家都设有专门的反恐机构，就互换情报，技术升级等各个层面都有定期的沟通渠道。就恐怖主义本身来讲，其产生的根源值得探讨。巴基斯坦现在政治局势也不稳定，经济也比较差，在这样的情况下，容易诱发恐怖主义活动苗头。不少境外势力和情报组织都已经渗透到了巴基斯坦国内，例如CIA等，他们可能通过收买穷人良心的方式实施破坏性活动，我们需要一个针对性的综合性的解决方案。\n\n首先就是要不断提升国民经济水平，保证每个人都能吃饱饭，铲除因贫困而铤而走险的思想土壤。很多事实都一再证明了贫穷和恐怖主义有紧密的相关性。\n\n观察者网：中巴关系历久弥新。但前任巴基斯坦总理伊姆兰·汗施政方针曾被国内反对派批评较多。巴基斯坦确实存在不同的政治群体和利益集团，如何看这一点对两国关系的影响？\n\n扎米尔·艾哈迈德·阿旺：新中国成立以来，历任领导人都有自己的历史使命，要面临不同时代赋予的历史责任，但不管最高领导人是谁，都和巴基斯坦保持着高度互信的外交关系。但巴基斯坦最高层领导的更替影响了内政方针实施的连续性，而且领导人的更换也比中国要频繁的多，但我们必须要认识到，无论政坛高层人事布局如何变，执政党派如何变，但中巴老百姓之间的利益不会变，中巴友谊也不会变，这是个原则性问题。\n\n巴基斯坦顶层设计经常有变动，多党制制度下，每个执政党上台后也都面临政绩压力，和在野党会出现各种党争，这种现象恐怕短期内暂时无法改变。\n\n我相信最高领导职位的变动对两个国家间的感情不会造成什么影响。'

In [27]:
nodes[3].id_

'2fbb07f2-9d8f-4409-bb88-1246d3de0c9a'